<a href="https://colab.research.google.com/github/SZ330/EE344-Stock-Market-Predictor/blob/main/SPY_Predictors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SPY Predictors for 1995 - 2025

Step 1: Import libraries

In [22]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import classification_report, confusion_matrix

from sklearn.preprocessing import StandardScaler
# from sklearn.pipeline import Pipeline (May need it, not using it for now)

Step 2: Load data set and drop columns we don't need

In [27]:
# Read the file path
CSV_PATH = 'SPY_1995_2025_feature_expansion.csv'
df = pd.read_csv(CSV_PATH)

df = df[df['y_next_dir'] != 0].reset_index(drop=True)
df['y_next_dir'].value_counts()

# Drop the date column
df_new = df.drop(columns=['Open', 'High', 'Low', 'Close',
                          'dom', 'weekofyear', 'ret_1d_lag6', 'ret_1d_lag7',
                          'ret_1d_lag8', 'ret_1d_lag9', 'ret_1d_lag10',
                          'ma_close_5d', 'ma_close_10d', 'ma_close_20d',
                          'ma_close_50d', 'close_minus_ma_5d', 'close_minus_ma_10d',
                          'close_minus_ma_20d', 'close_minus_ma_50d', 'vol_z_5d',
                          'vol_z_20d'])

# Print statements for sanity check
print(df_new.head())
print(df_new.tail())
print(df_new.shape)

         Date  Volume  dow  month    ret_1d  ret_1d_lag1  ret_1d_lag2  \
0  1995-03-14  223300    1      3  0.007302    -0.000951     0.014805   
1  1995-03-15  278500    2      3 -0.001891     0.007302    -0.000951   
2  1995-03-16   20400    3      3  0.005999    -0.001891     0.007302   
3  1995-03-20   91700    0      3  0.000000    -0.004394     0.005999   
4  1995-03-21  104400    1      3 -0.002522     0.000000    -0.004394   

   ret_1d_lag3  ret_1d_lag4  ret_1d_lag5  ...  close_over_ma_10d  \
0    -0.000322     0.002581    -0.007682  ...           1.014548   
1     0.014805    -0.000322     0.002581  ...           1.011014   
2    -0.000951     0.014805    -0.000322  ...           1.014973   
3    -0.001891     0.007302    -0.000951  ...           1.007368   
4     0.005999    -0.001891     0.007302  ...           1.002789   

   close_over_ma_20d  close_over_ma_50d    mom_5d   mom_10d   mom_20d  \
0           1.016319           1.039040  0.023548  0.011476  0.024871   
1     

Step 3: Split the data into 15 year of train and 15 years of test

In [36]:
# Explicitly mention features and target
FEATURE_COLS = ['Volume','dow','month','ret_1d','ret_1d_lag1',
                 'ret_1d_lag2','ret_1d_lag3','ret_1d_lag4','ret_1d_lag5',
                 'ret_vol_5d','ret_vol_10d','ret_vol_20d','close_over_ma_5d',
                 'close_over_ma_10d','close_over_ma_20d','close_over_ma_50d',
                 'mom_5d','mom_10d','mom_20d','vol_chg_1d','vol_mean_5d',
                 'vol_mean_20d']
TARGET_COL = "y_next_dir"

X = df_new[FEATURE_COLS]
y = df_new[TARGET_COL]

# Split train and test data down the middle (2010)
split_date = "1/4/2010"
train_df = df_new[df_new["Date"] < split_date]
test_df  = df_new[df_new["Date"] >= split_date]

# Sanity check
print("TRAIN RANGE:")
print(train_df["Date"].min(), "to", train_df["Date"].max())

print("\nTEST RANGE:")
print(test_df["Date"].min(), "to", test_df["Date"].max())

TRAIN RANGE:
1995-03-14 00:00:00 to 2009-12-31 00:00:00

TEST RANGE:
2010-01-04 00:00:00 to 2025-12-29 00:00:00
